<a href="https://colab.research.google.com/github/hrnnikolov/ML_models/blob/main/LSTM_price_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
import torch
import torch.nn as nn

data = yf.download(tickers='AAPL', start='2014-01-01', end='2024-02-20')
data.head(10)

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,19.845715,19.893929,19.715000,19.754642,17.296659,234684800
2014-01-03,19.745001,19.775000,19.301071,19.320715,16.916719,392467600
2014-01-06,19.194643,19.528570,19.057142,19.426071,17.008970,412610800
2014-01-07,19.440001,19.498571,19.211430,19.287144,16.887323,317209200
2014-01-08,19.243214,19.484285,19.238930,19.409286,16.994274,258529600
2014-01-09,19.528570,19.530714,19.119642,19.161428,16.777254,279148800
2014-01-10,19.279642,19.314285,18.968214,19.033571,16.665306,304976000
2014-01-13,18.925358,19.375000,18.924286,19.133215,16.752546,378492800
2014-01-14,19.222143,19.526072,19.202143,19.513929,17.085894,332561600


In [2]:
data['RSI'] = ta.rsi(data.Close, length=15)
data['EMAF'] = ta.ema(data.Close, length=20)
data['EMAM'] = ta.ema(data.Close, length=100)
data['EMAS'] = ta.ema(data.Close, length=150)

data['Target'] = data['Adj Close'] - data.Open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i] > 0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)

data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

In [3]:
data

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
data_set = data.iloc[:, 0:11]
pd.set_option('display.max_columns', None)

data_set.head(20)

,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
0,23.687500,23.870001,23.677500,21.027348,50.075969,23.903334,22.073109,20.815405,-2.707371,0,21.025129
1,23.732500,23.987499,23.525000,21.025129,47.998875,23.876350,22.103741,20.852552,-2.482023,0,21.082977
2,23.565001,23.705000,23.320000,21.082977,49.221234,23.858126,22.135053,20.890068,-2.456337,0,21.361162
3,23.817499,24.020000,23.709999,21.361162,54.705752,23.871400,22.171933,20.931226,-2.653296,0,21.356705
4,24.010000,24.219999,23.902500,21.356705,54.604665,23.882933,22.207984,20.971772,-2.398180,0,21.639320
5,24.037500,24.309999,24.010000,21.639320,59.675102,23.923606,22.249608,21.015987,-2.635321,0,21.697180
6,24.332500,24.392500,24.200001,21.697180,60.639448,23.966596,22.291695,21.060478,-2.671003,0,21.803997
7,24.475000,24.547501,24.215000,21.803997,62.417232,24.016920,22.335325,21.105968,-2.555914,0,22.066586
8,24.622499,24.842501,24.495001,22.066586,66.412916,24.090547,22.383932,21.154763,-2.481047,0,22.371454
9,24.852501,25.170000,24.830000,22.371454,70.336036,24.189781,22.438359,21.207448,-2.729639,0,22.380362


In [5]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.00676561 0.00539042 0.00750146 ... 0.36994643 0.         0.00142853]
 [0.00702199 0.00605537 0.0066282  ... 0.38282029 0.         0.0017552 ]
 [0.00606769 0.00445665 0.00545431 ... 0.38428772 0.         0.0033261 ]
 ...
 [0.93020739 0.9241098  0.92275203 ... 0.45777398 0.         0.92259364]
 [0.92764359 0.92026148 0.91662493 ... 0.54232539 1.         0.92095605]
 [0.91755925 0.91437591 0.91038327 ... 0.46120245 0.         0.91220319]]


In [6]:
# X = []

# backcandles = 30 #how many days to look back
# print(data_set_scaled.shape[0])

# for j in range(8):
#   X.append([])
#   for i in range(backcandles, data_set_scaled.shape[0]):
#     X[j].append(data_set_scaled[i-backcandles: i, j])

# #move axis from 0 to position 2
# X=np.moveaxis(X, [0], [2])

# # Choose -1 for last column, classification else -2...
# X, yi = np.array(X), np.array(data_set_scaled[backcandles:, -3])
# y=np.reshape(yi, (len(yi), 1))

# print(X.shape)
# print(y.shape)


In [7]:
splitlimit = int(len(X)*0.8)

X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]

print(X_train.shape, y_test.shape)

NameError: name 'X' is not defined

In [ ]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [ ]:
train_dataset

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

In [ ]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [ ]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [ ]:
learning_rate = 0.001
num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color='black', label='Test')
plt.plot(y_pred, color='green', label='Pred')
plt.legend()
plt.show()